In [ ]:
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import tritonclient.grpc as grpcclient
from tqdm.auto import tqdm
from dotenv import load_dotenv
import random
import ipyplot

from utils import *

load_dotenv()

In [ ]:
with open(JSON_DATA_FILE) as f:
    facedetect_data = json.load(f)

In [ ]:
triton_client = grpcclient.InferenceServerClient(url=TRITON_SERVER_URL, verbose=False)

model_metadata = triton_client.get_model_metadata(
    model_name=FPENET_MODEL_NAME, model_version=MODEL_VERSION
)

model_config = triton_client.get_model_config(
    model_name=FPENET_MODEL_NAME, model_version=MODEL_VERSION
).config

input_names = [i.name for i in model_config.input]
output_names = [i.name for i in model_config.output]

In [ ]:
def render_fpenet_image(clip, points):
    img = Image.fromarray(clip.reshape((80, 80)).astype("uint8"), "L").convert("RGB")
    draw = ImageDraw.Draw(img)
    for x, y in points:
        draw.point((x, y), fill="#76b900")
    return img

In [ ]:
n = 0
max_images = 8
show_me_grid = []
results = []
for json_items in facedetect_data:
    if (json_items["faces"][0]["probability"] != 0) and (n <= max_images):
        try:
            results.append(
                submit_to_fpenet(
                    json_items,
                    input_names[0],
                    output_names,
                    request_id=json_items["filename"],
                    timeit=False,
                    return_clips=True,
                )
            )
            n += 1
        except Exception as e:
            print("There was an exception: {}".format(e))


for responses, clips in results:
    for response, clip in zip(responses, clips):
        points = response.squeeze()
        img = render_fpenet_image(clip, points)
        show_me_grid.append(img)

ipyplot.plot_images(
    images=show_me_grid, img_width=160, show_url=False, max_images=max_images
)

In [ ]:
# DEBUG
# facedetect_data = random.sample(facedetect_data, N_DEBUG)

In [ ]:
pbar = tqdm(
    total=len(facedetect_data),
    desc="Submitting photos to {} at {}".format(FPENET_MODEL_NAME, TRITON_SERVER_URL),
)

futures = []
results = []

with ThreadPoolExecutor() as executor:
    for chunk in chunked(facedetect_data, THREAD_CHUNKS):
        futures = []
        for json_items in chunk:
            pbar.update()
            if json_items["faces"][0]["probability"] != 0:
                futures.append(
                    executor.submit(
                        submit_to_fpenet,
                        json_items,
                        input_names[0],
                        output_names,
                        request_id=json_items["filename"],
                    )
                )

        for future in as_completed(futures):
            try:
                infer_result = future.result()
                results.append(infer_result)
            except Exception as e:
                print("There was an exception: {}.".format(e))

___